In [1]:
import os
import sys
sys.path.append('../')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import jax
import jax.numpy as jnp
from flax import jax_utils
from flax.training import checkpoints
from tqdm import tqdm
from functools import partial

from giung2.config import get_cfg
from giung2.data.build import build_dataloaders
from giung2.modeling.build import build_model
from giung2.evaluation import get_optimal_temperature, temperature_scaling, evaluate_acc, evaluate_nll, evaluate_ece

CPU = jax.devices("cpu")[0]

In [3]:
# initialize configuration
cfg = get_cfg()
cfg.merge_from_file("../configs/C100_WRN28x4.yaml", allow_unsafe=True)
cfg.DATASETS.ROOT = "../datasets/"

In [4]:
# build model & dataloaders
model = build_model(cfg)
rng = jax.random.PRNGKey(0)

def initialize_model(key, model, im_shape, im_dtype):
    @jax.jit
    def init(*args):
        return model.init(*args)
    var_dict = init({'params': key}, jnp.ones(im_shape, im_dtype))
    return var_dict

if cfg.DATASETS.NAME in ['CIFAR10',]:
    image_shape = (1, 32, 32, 3,)
    num_classes = 10
elif cfg.DATASETS.NAME in ['CIFAR100',]:
    image_shape = (1, 32, 32, 3,)
    num_classes = 100

im_dtype = jnp.float32
initialize_model(rng, model, image_shape, im_dtype)

dataloaders = build_dataloaders(cfg, batch_size=200)

### Teacher (DE-4)

In [5]:
# load pre-trained weights
teacher_var_dict_list = []
for idx in range(4):
    ckpt = checkpoints.restore_checkpoint(
        f"../outputs/C100_WRN28x4/SGD/{idx}/best_acc1", target=None)
    teacher_var_dict_list.append({
        'params': ckpt['params'],
        'image_stats': ckpt['image_stats'],
        'batch_stats': ckpt['batch_stats'],})
    del ckpt

# make predictions
def predict(var_dict_list, images):
    return jnp.stack([
        model.apply(
            var_dict, images, mutable='intermediates'
        )[1]['intermediates']['classifier']['logits'][0]
        for var_dict in var_dict_list
    ], axis=1) # [N, M, K]

predict = jax.pmap(partial(predict, teacher_var_dict_list), axis_name='batch')

def make_predictions(dataloader, desc):
    true_labels, pred_logits = [], []
    for batch in tqdm(dataloader, desc=desc, leave=False):
        labels = jax.device_put(jnp.concatenate(        batch['labels'] ), CPU)
        logits = jax.device_put(jnp.concatenate(predict(batch['images'])), CPU)
        true_labels.append(labels)
        pred_logits.append(logits)
    return jnp.concatenate(true_labels), jnp.concatenate(pred_logits)

val_true_labels, val_pred_logits = make_predictions(
    jax_utils.prefetch_to_device(dataloaders['val_loader'](rng=None), size=2),
    'Make predictions on valid examples')
tst_true_labels, tst_pred_logits = make_predictions(
    jax_utils.prefetch_to_device(dataloaders['tst_loader'](rng=None), size=2),
    'Make predictions on test examples')

# evaluate predictions
val_confidences = jnp.mean(jax.nn.softmax(val_pred_logits, axis=-1), axis=1) # [N, K,]
tst_confidences = jnp.mean(jax.nn.softmax(tst_pred_logits, axis=-1), axis=1) # [N, K,]
t_opt = get_optimal_temperature(val_confidences, val_true_labels, log_input=False)
print('| ACC    | NLL    | ECE    | cNLL   | cECE   |')
print('| {:.2f}  | {:.3f}  | {:.3f}  | {:.3f}  | {:.3f}  |'.format(
    evaluate_acc(                    tst_confidences,                          tst_true_labels, log_input=False) * 100,
    evaluate_nll(                    tst_confidences,                          tst_true_labels, log_input=False),
    evaluate_ece(                    tst_confidences,                          tst_true_labels, log_input=False)['ece'],
    evaluate_nll(temperature_scaling(tst_confidences, t_opt, log_input=False), tst_true_labels, log_input=False),
    evaluate_ece(temperature_scaling(tst_confidences, t_opt, log_input=False), tst_true_labels, log_input=False)['ece'],
))

| ACC    | NLL    | ECE    | cNLL   | cECE   |
| 81.08  | 0.715  | 0.030  | 0.709  | 0.018  |


### Student (KD)

In [6]:
NUM_EXPERIMENTS = 4

In [7]:
# load pre-trained weights
student_var_dict_list = []
for idx in range(NUM_EXPERIMENTS):
    ckpt = checkpoints.restore_checkpoint(
        f"../outputs/C100_WRN28x4/KD_DE4/{idx}/best_acc1", target=None)
    student_var_dict_list.append({
        'params': ckpt['params'],
        'image_stats': ckpt['image_stats'],
        'batch_stats': ckpt['batch_stats'],})
    del ckpt

# make predictions
def predict(var_dict_list, images):
    return jnp.stack([
        model.apply(
            var_dict, images, mutable='intermediates'
        )[1]['intermediates']['classifier']['logits'][0]
        for var_dict in var_dict_list
    ], axis=1) # [N, M, K]

predict = jax.pmap(partial(predict, student_var_dict_list), axis_name='batch')

def make_predictions(dataloader, desc):
    true_labels, pred_logits = [], []
    for batch in tqdm(dataloader, desc=desc, leave=False):
        labels = jax.device_put(jnp.concatenate(        batch['labels'] ), CPU)
        logits = jax.device_put(jnp.concatenate(predict(batch['images'])), CPU)
        true_labels.append(labels)
        pred_logits.append(logits)
    return jnp.concatenate(true_labels), jnp.concatenate(pred_logits)

val_true_labels, val_pred_logits = make_predictions(
    jax_utils.prefetch_to_device(dataloaders['val_loader'](rng=None), size=2),
    'Make predictions on valid examples')
tst_true_labels, tst_pred_logits = make_predictions(
    jax_utils.prefetch_to_device(dataloaders['tst_loader'](rng=None), size=2),
    'Make predictions on test examples')

# evaluate predictions
print('| ACC    | NLL    | ECE    | cNLL   | cECE   |')
for idx in range(NUM_EXPERIMENTS):
    val_confidences = jax.nn.softmax(val_pred_logits, axis=-1)[:, idx, :] # [N, K,]
    tst_confidences = jax.nn.softmax(tst_pred_logits, axis=-1)[:, idx, :] # [N, K,]
    t_opt = get_optimal_temperature(val_confidences, val_true_labels, log_input=False)
    print('| {:.2f}  | {:.3f}  | {:.3f}  | {:.3f}  | {:.3f}  |'.format(
        evaluate_acc(                    tst_confidences,                          tst_true_labels, log_input=False) * 100,
        evaluate_nll(                    tst_confidences,                          tst_true_labels, log_input=False),
        evaluate_ece(                    tst_confidences,                          tst_true_labels, log_input=False)['ece'],
        evaluate_nll(temperature_scaling(tst_confidences, t_opt, log_input=False), tst_true_labels, log_input=False),
        evaluate_ece(temperature_scaling(tst_confidences, t_opt, log_input=False), tst_true_labels, log_input=False)['ece'],
    ))

| ACC    | NLL    | ECE    | cNLL   | cECE   |
| 79.00  | 1.064  | 0.136  | 0.865  | 0.041  |
| 79.53  | 1.064  | 0.129  | 0.867  | 0.044  |
| 79.11  | 1.081  | 0.135  | 0.873  | 0.045  |
| 79.03  | 1.080  | 0.135  | 0.877  | 0.044  |


### Student (KD + LatentBE)

In [8]:
# load pre-trained weights
student_var_dict_list = []
for idx in range(NUM_EXPERIMENTS):
    ckpt = checkpoints.restore_checkpoint(
        f"../outputs/C100_WRN28x4/KD_DE4_LatentBE/{idx}/best_acc1", target=None)
    
    # merge rank-one factors
    ckpt['params']['classifier']['Linear_BatchEnsemble_0']['w'] *= jnp.expand_dims(
        jnp.mean(ckpt['params']['classifier']['Linear_BatchEnsemble_0']['r'], axis=0), axis=1)
    ckpt['params']['classifier']['Linear_BatchEnsemble_0']['w'] *= jnp.expand_dims(
        jnp.mean(ckpt['params']['classifier']['Linear_BatchEnsemble_0']['s'], axis=0), axis=0)
    del ckpt['params']['classifier']['Linear_BatchEnsemble_0']['r']
    del ckpt['params']['classifier']['Linear_BatchEnsemble_0']['s']
    ckpt['params']['classifier']['Linear_0'] = ckpt['params']['classifier']['Linear_BatchEnsemble_0']
    del ckpt['params']['classifier']['Linear_BatchEnsemble_0']
    
    ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']['w'] *= jnp.expand_dims(
        jnp.mean(ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']['r'], axis=0), axis=(0, 1, 3,))
    ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']['w'] *= jnp.expand_dims(
        jnp.mean(ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']['s'], axis=0), axis=(0, 1, 2,))
    del ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']['r']
    del ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']['s']
    ckpt['params']['backbone']['FirstBlock_0']['Conv2d_0'] = ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']
    del ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']
    
    for block_name in [f'BasicBlock_{jdx}' for jdx in range(12)]:
        ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']['w'] *= jnp.expand_dims(
            jnp.mean(ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']['r'], axis=0), axis=(0, 1, 3,))
        ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']['w'] *= jnp.expand_dims(
            jnp.mean(ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']['s'], axis=0), axis=(0, 1, 2,))
        del ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']['r']
        del ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']['s']
        ckpt['params']['backbone'][block_name]['Conv2d_0'] = ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']
        del ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']
        
        ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']['w'] *= jnp.expand_dims(
            jnp.mean(ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']['r'], axis=0), axis=(0, 1, 3,))
        ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']['w'] *= jnp.expand_dims(
            jnp.mean(ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']['s'], axis=0), axis=(0, 1, 2,))
        del ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']['r']
        del ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']['s']
        ckpt['params']['backbone'][block_name]['Conv2d_1'] = ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']
        del ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']
    
    for block_name in [f'BasicBlock_{jdx}' for jdx in [0, 4, 8,]]:
        ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']['w'] *= jnp.expand_dims(
            jnp.mean(ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']['r'], axis=0), axis=(0, 1, 3,))
        ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']['w'] *= jnp.expand_dims(
            jnp.mean(ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']['s'], axis=0), axis=(0, 1, 2,))
        del ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']['r']
        del ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']['s']
        ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_0'] = ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']
        del ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']
    
    student_var_dict_list.append({
        'params': ckpt['params'],
        'image_stats': ckpt['image_stats'],
        'batch_stats': ckpt['batch_stats'],})
    
    del ckpt

# make predictions
def predict(var_dict_list, images):
    return jnp.stack([
        model.apply(
            var_dict, images, mutable='intermediates'
        )[1]['intermediates']['classifier']['logits'][0]
        for var_dict in var_dict_list
    ], axis=1) # [N, M, K]

predict = jax.pmap(partial(predict, student_var_dict_list), axis_name='batch')

def make_predictions(dataloader, desc):
    true_labels, pred_logits = [], []
    for batch in tqdm(dataloader, desc=desc, leave=False):
        labels = jax.device_put(jnp.concatenate(        batch['labels'] ), CPU)
        logits = jax.device_put(jnp.concatenate(predict(batch['images'])), CPU)
        true_labels.append(labels)
        pred_logits.append(logits)
    return jnp.concatenate(true_labels), jnp.concatenate(pred_logits)

val_true_labels, val_pred_logits = make_predictions(
    jax_utils.prefetch_to_device(dataloaders['val_loader'](rng=None), size=2),
    'Make predictions on valid examples')
tst_true_labels, tst_pred_logits = make_predictions(
    jax_utils.prefetch_to_device(dataloaders['tst_loader'](rng=None), size=2),
    'Make predictions on test examples')

# evaluate predictions
print('| ACC    | NLL    | ECE    | cNLL   | cECE   |')
for idx in range(NUM_EXPERIMENTS):
    val_confidences = jax.nn.softmax(val_pred_logits, axis=-1)[:, idx, :] # [N, K,]
    tst_confidences = jax.nn.softmax(tst_pred_logits, axis=-1)[:, idx, :] # [N, K,]
    t_opt = get_optimal_temperature(val_confidences, val_true_labels, log_input=False)
    print('| {:.2f}  | {:.3f}  | {:.3f}  | {:.3f}  | {:.3f}  |'.format(
        evaluate_acc(                    tst_confidences,                          tst_true_labels, log_input=False) * 100,
        evaluate_nll(                    tst_confidences,                          tst_true_labels, log_input=False),
        evaluate_ece(                    tst_confidences,                          tst_true_labels, log_input=False)['ece'],
        evaluate_nll(temperature_scaling(tst_confidences, t_opt, log_input=False), tst_true_labels, log_input=False),
        evaluate_ece(temperature_scaling(tst_confidences, t_opt, log_input=False), tst_true_labels, log_input=False)['ece'],
    ))

| ACC    | NLL    | ECE    | cNLL   | cECE   |
| 79.03  | 1.024  | 0.130  | 0.850  | 0.045  |
| 79.48  | 0.999  | 0.125  | 0.838  | 0.045  |
| 79.41  | 1.014  | 0.128  | 0.842  | 0.046  |
| 79.37  | 1.010  | 0.126  | 0.843  | 0.046  |


### Student (KD + LatentBE + TDiv-SDiv)

In [9]:
# load pre-trained weights
student_var_dict_list = []
for idx in range(NUM_EXPERIMENTS):
    ckpt = checkpoints.restore_checkpoint(
        f"../outputs/C100_WRN28x4/KD_DE4_LatentBE_TDivSDiv/{idx}/best_acc1", target=None)
    
    # merge rank-one factors
    ckpt['params']['classifier']['Linear_BatchEnsemble_0']['w'] *= jnp.expand_dims(
        jnp.mean(ckpt['params']['classifier']['Linear_BatchEnsemble_0']['r'], axis=0), axis=1)
    ckpt['params']['classifier']['Linear_BatchEnsemble_0']['w'] *= jnp.expand_dims(
        jnp.mean(ckpt['params']['classifier']['Linear_BatchEnsemble_0']['s'], axis=0), axis=0)
    del ckpt['params']['classifier']['Linear_BatchEnsemble_0']['r']
    del ckpt['params']['classifier']['Linear_BatchEnsemble_0']['s']
    ckpt['params']['classifier']['Linear_0'] = ckpt['params']['classifier']['Linear_BatchEnsemble_0']
    del ckpt['params']['classifier']['Linear_BatchEnsemble_0']
    
    ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']['w'] *= jnp.expand_dims(
        jnp.mean(ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']['r'], axis=0), axis=(0, 1, 3,))
    ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']['w'] *= jnp.expand_dims(
        jnp.mean(ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']['s'], axis=0), axis=(0, 1, 2,))
    del ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']['r']
    del ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']['s']
    ckpt['params']['backbone']['FirstBlock_0']['Conv2d_0'] = ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']
    del ckpt['params']['backbone']['FirstBlock_0']['Conv2d_BatchEnsemble_0']
    
    for block_name in [f'BasicBlock_{jdx}' for jdx in range(12)]:
        ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']['w'] *= jnp.expand_dims(
            jnp.mean(ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']['r'], axis=0), axis=(0, 1, 3,))
        ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']['w'] *= jnp.expand_dims(
            jnp.mean(ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']['s'], axis=0), axis=(0, 1, 2,))
        del ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']['r']
        del ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']['s']
        ckpt['params']['backbone'][block_name]['Conv2d_0'] = ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']
        del ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_0']
        
        ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']['w'] *= jnp.expand_dims(
            jnp.mean(ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']['r'], axis=0), axis=(0, 1, 3,))
        ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']['w'] *= jnp.expand_dims(
            jnp.mean(ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']['s'], axis=0), axis=(0, 1, 2,))
        del ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']['r']
        del ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']['s']
        ckpt['params']['backbone'][block_name]['Conv2d_1'] = ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']
        del ckpt['params']['backbone'][block_name]['Conv2d_BatchEnsemble_1']
    
    for block_name in [f'BasicBlock_{jdx}' for jdx in [0, 4, 8,]]:
        ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']['w'] *= jnp.expand_dims(
            jnp.mean(ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']['r'], axis=0), axis=(0, 1, 3,))
        ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']['w'] *= jnp.expand_dims(
            jnp.mean(ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']['s'], axis=0), axis=(0, 1, 2,))
        del ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']['r']
        del ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']['s']
        ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_0'] = ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']
        del ckpt['params']['backbone'][block_name]['ProjectionShortcut_0']['Conv2d_BatchEnsemble_0']
    
    student_var_dict_list.append({
        'params': ckpt['params'],
        'image_stats': ckpt['image_stats'],
        'batch_stats': ckpt['batch_stats'],})
    
    del ckpt

# make predictions
def predict(var_dict_list, images):
    return jnp.stack([
        model.apply(
            var_dict, images, mutable='intermediates'
        )[1]['intermediates']['classifier']['logits'][0]
        for var_dict in var_dict_list
    ], axis=1) # [N, M, K]

predict = jax.pmap(partial(predict, student_var_dict_list), axis_name='batch')

def make_predictions(dataloader, desc):
    true_labels, pred_logits = [], []
    for batch in tqdm(dataloader, desc=desc, leave=False):
        labels = jax.device_put(jnp.concatenate(        batch['labels'] ), CPU)
        logits = jax.device_put(jnp.concatenate(predict(batch['images'])), CPU)
        true_labels.append(labels)
        pred_logits.append(logits)
    return jnp.concatenate(true_labels), jnp.concatenate(pred_logits)

val_true_labels, val_pred_logits = make_predictions(
    jax_utils.prefetch_to_device(dataloaders['val_loader'](rng=None), size=2),
    'Make predictions on valid examples')
tst_true_labels, tst_pred_logits = make_predictions(
    jax_utils.prefetch_to_device(dataloaders['tst_loader'](rng=None), size=2),
    'Make predictions on test examples')

# evaluate predictions
print('| ACC    | NLL    | ECE    | cNLL   | cECE   |')
for idx in range(NUM_EXPERIMENTS):
    val_confidences = jax.nn.softmax(val_pred_logits, axis=-1)[:, idx, :] # [N, K,]
    tst_confidences = jax.nn.softmax(tst_pred_logits, axis=-1)[:, idx, :] # [N, K,]
    t_opt = get_optimal_temperature(val_confidences, val_true_labels, log_input=False)
    print('| {:.2f}  | {:.3f}  | {:.3f}  | {:.3f}  | {:.3f}  |'.format(
        evaluate_acc(                    tst_confidences,                          tst_true_labels, log_input=False) * 100,
        evaluate_nll(                    tst_confidences,                          tst_true_labels, log_input=False),
        evaluate_ece(                    tst_confidences,                          tst_true_labels, log_input=False)['ece'],
        evaluate_nll(temperature_scaling(tst_confidences, t_opt, log_input=False), tst_true_labels, log_input=False),
        evaluate_ece(temperature_scaling(tst_confidences, t_opt, log_input=False), tst_true_labels, log_input=False)['ece'],
    ))

| ACC    | NLL    | ECE    | cNLL   | cECE   |
| 79.98  | 0.808  | 0.071  | 0.777  | 0.044  |
| 80.21  | 0.799  | 0.070  | 0.771  | 0.044  |
| 79.99  | 0.792  | 0.074  | 0.764  | 0.037  |
| 80.16  | 0.793  | 0.067  | 0.768  | 0.038  |
